<a href="https://colab.research.google.com/github/dijkstra001/ia-bot-trader/blob/main/IA_bot_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IA Bot Trader**

Esse projeto tem como objetivo desenvolver uma IA para realizar recomendações de compra e venda de criptomoedas.

Os dados utilizados são da Exchange Bitmex e a API utilizada está disponível no link: https://www.bitmex.com/app/apiOverview

***Obs: Ao utilizar API para requisição dos dados, cuidado para não exceder o número de requisições.***

In [ ]:
!pip install -q ccxt

In [ ]:
!pip install -q bayesian-optimization==1.2

In [6]:
import csv
import ccxt
import time
import random
import types
import pkg_resources
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
from bayes_opt import BayesianOptimization
from google.colab import drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def generate_csv(archive, data):
  with open(archive, mode = 'w') as archive_csv:
    archive_csv.write("Date,Open,High,Low,Close,Adj Close,Volume\n")

    csv_writer = csv.writer(archive_csv, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerows(data)

In [10]:
def exchange_connection(exchange, max_retries, symbol, timeframe, since, limit):
  num_retries = 0

  try:
    num_retries += 1
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
    return ohlcv

  except Exception:
      if num_retries > max_retries:
        raise

In [11]:
def extract_data(exchange, max_retries, symbol, timeframe, since, limit):
  earliest_timestamp = exchange.milliseconds()

  timeframe_duration_in_sec = exchange.parse_timeframe(timeframe)
  timeframe_duration_in_ms = timeframe_duration_in_sec * 1000
  timedelta = limit * timeframe_duration_in_ms

  all_ohlcv = []

  while True:
    fetch_since = earliest_timestamp - timedelta
    ohlcv = exchange_connection(exchange, max_retries, symbol, timeframe, fetch_since, limit)

    if ohlcv[0][0] >= earliest_timestamp:
      break
    earliest_timestamp = ohlcv[0][0]
    all_ohlcv = ohlcv + all_ohlcv
    print(f'{len(all_ohlcv)} registros extraídos de {exchange.iso8601(all_ohlcv[0][0])} a {exchange.iso8601(all_ohlcv[-1][0])}')

    if fetch_since < since:
      break
  
  return all_ohlcv

In [12]:
def extract_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True,})
    
    if isinstance(since, str):
        since = exchange.parse8601(since)
    exchange.load_markets()
    
    ohlcv = extract_data(exchange, max_retries, symbol, timeframe, since, limit)
    key = 0
    
    for item in ohlcv:
        epoch = int(item[0]) / 1000
        ohlcv[key][0] = datetime.utcfromtimestamp(epoch).strftime('%Y-%m-%d')
        ohlcv[key][5] = int(item[5])
        ohlcv[key].append(ohlcv[key][5])
        ohlcv[key][5] = ohlcv[key][4]
        key += 1
    ohlen = len(ohlcv)
    pprint("Número de Registros: " + str(ohlen))
    
    """if ohlen > 399:
        ohrem = ohlen - 399
        pprint("Removendo: " + str(ohrem))
        ohlcv = ohlcv[ohrem:]""" # Se não tiver recursos suficientes (ram e memória) descomente esse código
    
    generate_csv(filename, ohlcv)
    print('Salvos', len(ohlcv), 'registros no arquivo', filename)

In [17]:
# Exchange: https://www.bitmex.com/app/apiOverview
exchange = "bitmex"
symbol = "BTC/USD"
timeframe = "1d"
since = "2018-01-01T00:00:00Z"
max_retries = 3
outfile = "/content/drive/My Drive/ia-bot-trader/dataset.csv" # Aponte para o diretório onde você deseja salvar os dados

In [18]:
extract_to_csv(outfile, exchange, max_retries, symbol, timeframe, since, 100)

100 registros extraídos de 2021-02-01T00:00:00.000Z a 2021-05-11T00:00:00.000Z
200 registros extraídos de 2020-10-24T00:00:00.000Z a 2021-05-11T00:00:00.000Z
300 registros extraídos de 2020-07-16T00:00:00.000Z a 2021-05-11T00:00:00.000Z
400 registros extraídos de 2020-04-07T00:00:00.000Z a 2021-05-11T00:00:00.000Z
500 registros extraídos de 2019-12-29T00:00:00.000Z a 2021-05-11T00:00:00.000Z
600 registros extraídos de 2019-09-20T00:00:00.000Z a 2021-05-11T00:00:00.000Z
700 registros extraídos de 2019-06-12T00:00:00.000Z a 2021-05-11T00:00:00.000Z
800 registros extraídos de 2019-03-04T00:00:00.000Z a 2021-05-11T00:00:00.000Z
900 registros extraídos de 2018-11-24T00:00:00.000Z a 2021-05-11T00:00:00.000Z
1000 registros extraídos de 2018-08-16T00:00:00.000Z a 2021-05-11T00:00:00.000Z
1100 registros extraídos de 2018-05-08T00:00:00.000Z a 2021-05-11T00:00:00.000Z
1200 registros extraídos de 2018-01-28T00:00:00.000Z a 2021-05-11T00:00:00.000Z
1300 registros extraídos de 2017-10-20T00:00:00.0

# **Análise Exploratória:**

In [29]:
dataset = pd.read_csv(outfile)
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-20,5700.8,6168.9,5602.0,6017.5,6017.5,912558192
1,2017-10-21,6017.5,6220.0,5875.0,6018.7,6018.7,854248299
2,2017-10-22,6018.7,6080.2,5720.0,5986.0,5986.0,847520369
3,2017-10-23,5986.0,6095.2,5638.7,5853.9,5853.9,1013064819
4,2017-10-24,5853.9,5853.9,5454.0,5517.4,5517.4,950744064


In [23]:
dataset.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1300.000000,1300.000000,1300.000000,1300.000000,1300.000000,1.300000e+03
mean,12800.989000,13213.377000,12353.268154,12839.603769,12839.603769,2.309433e+09
std,12933.095774,13372.277615,12447.087131,12986.709760,12986.709760,1.421668e+09
min,3183.000000,3226.000000,3121.000000,3183.000000,3183.000000,3.202566e+08
25%,6670.625000,6818.000000,6465.375000,6682.000000,6682.000000,1.305385e+09
50%,8797.000000,9020.000000,8539.000000,8803.000000,8803.000000,2.060543e+09
75%,11160.625000,11512.875000,10803.875000,11176.750000,11176.750000,2.906428e+09
max,63674.000000,65000.000000,62131.000000,63674.000000,63674.000000,1.362211e+10
